# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def retrieve_titles(soup):
    titles = []
    for h3 in soup.find('div', class_="alert alert-warning").nextSibling.nextSibling.findAll('h3'):
        titles.append(h3.find('a').attrs['title'])
    return titles

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [3]:
def retrieve_ratings(soup):
    num_dic = {'One' : 1, 'Two':2, 'Three':3, 'Four':4, 'Five':5}
    regex = re.compile("star-rating (.*)")
    ratings = []
    for p in soup.find('div', class_="alert alert-warning").nextSibling.nextSibling.findAll('p', {"class" : regex}):
        ratings.append(num_dic[p.attrs['class'][-1]])
    return ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [4]:
def retrieve_prices(soup):
    prices=[]
    for pr in soup.find('div', class_="alert alert-warning").nextSibling.nextSibling.findAll('p', class_= 'price_color'):
        prices.append(float(pr.text[1:]))
    return prices

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [6]:
def retrieve_availabilities(soup):
    avail = []
    for av in soup.find('div', class_="alert alert-warning").nextSibling.nextSibling.findAll('p', class_= 'instock availability'):
        avail.append(av.text.strip())
    return avail

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [8]:
avail = []
prices=[]
ratings = []
titles = []

for i in range(1,51):
    if i == 1:
        url = 'http://books.toscrape.com'
    else:
        url = f'http://books.toscrape.com/catalogue/page-{i}.html'
       
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
   
    titles.extend(retrieve_titles(soup))
    print(url)
    ratings.extend(retrieve_ratings(soup))
    avail.extend(retrieve_availabilities(soup))
    prices.extend(retrieve_prices(soup))

df = pd.DataFrame([titles, prices, ratings, avail]).transpose()
df.columns=('title', 'price -£', 'rating', 'availability')
df

http://books.toscrape.com
http://books.toscrape.com/catalogue/page-2.html
http://books.toscrape.com/catalogue/page-3.html
http://books.toscrape.com/catalogue/page-4.html
http://books.toscrape.com/catalogue/page-5.html
http://books.toscrape.com/catalogue/page-6.html
http://books.toscrape.com/catalogue/page-7.html
http://books.toscrape.com/catalogue/page-8.html
http://books.toscrape.com/catalogue/page-9.html
http://books.toscrape.com/catalogue/page-10.html
http://books.toscrape.com/catalogue/page-11.html
http://books.toscrape.com/catalogue/page-12.html
http://books.toscrape.com/catalogue/page-13.html
http://books.toscrape.com/catalogue/page-14.html
http://books.toscrape.com/catalogue/page-15.html
http://books.toscrape.com/catalogue/page-16.html
http://books.toscrape.com/catalogue/page-17.html
http://books.toscrape.com/catalogue/page-18.html
http://books.toscrape.com/catalogue/page-19.html
http://books.toscrape.com/catalogue/page-20.html
http://books.toscrape.com/catalogue/page-21.html
ht

,title,price -£,rating,availability
0,A Light in the Attic,51.77,3,In stock
1,Tipping the Velvet,53.74,1,In stock
2,Soumission,50.1,1,In stock
3,Sharp Objects,47.82,4,In stock
4,Sapiens: A Brief History of Humankind,54.23,5,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,55.53,1,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,4,In stock
997,A Spy's Devotion (The Regency Spies of London #1),16.97,5,In stock
998,1st to Die (Women's Murder Club #1),53.98,1,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [9]:
avail = []
prices=[]
ratings = []
titles = []

#need to iterate through pages by finding href of next


def next_url(url_base, soup):
    if soup.find('li', class_='next'):
        href = soup.find('li', class_='next').find('a').attrs['href']
        if href[0] == 'p':
            url = url_base + "/catalogue/" + href
        else:
            url = url_base + "/" + href
        return url
    else:
        return None

def page_retriever(url):
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
   
   
    titles.extend(retrieve_titles(soup))
    ratings.extend(retrieve_ratings(soup))
    avail.extend(retrieve_availabilities(soup))
    prices.extend(retrieve_prices(soup))
   
   
    url = next_url(url_base, soup)
    if url:
        page_retriever(url)
    else:
        return titles, ratings, prices, avail

url_base = 'http://books.toscrape.com'
page_retriever(url=url_base)

df = pd.DataFrame([titles, prices, ratings, avail]).transpose()
df.columns=('title', 'price -£', 'rating', 'availability')
df

,title,price -£,rating,availability
0,A Light in the Attic,51.77,3,In stock
1,Tipping the Velvet,53.74,1,In stock
2,Soumission,50.1,1,In stock
3,Sharp Objects,47.82,4,In stock
4,Sapiens: A Brief History of Humankind,54.23,5,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,55.53,1,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,4,In stock
997,A Spy's Devotion (The Regency Spies of London #1),16.97,5,In stock
998,1st to Die (Women's Murder Club #1),53.98,1,In stock


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!